# Sentiment analysis project

In [223]:
# General
%matplotlib inline
import matplotlib.pyplot as plt
import os
from collections import Counter
from typing import List

# Keras
import keras
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.models import Sequential, load_model
from keras.layers import (Embedding, LSTM, Dense, Dropout,
                          Flatten, GRU, Conv1D, MaxPooling1D)

# Other
import nltk
from nltk.corpus import stopwords
import pandas as pd

## Load and preprocess the data

In [224]:
VOCAB_SIZE = 10000

In [225]:
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=VOCAB_SIZE)

### Mappers

In [226]:
class Preprocessing:
    def __init__(self):
        self.word2ind = imdb.get_word_index()
        self.ind2word = {ind: word for word, ind in self.word2ind.items()}
        
    def map_inds_to_words(self, inds: List) -> List:
        return [self.ind2word.get(ind, '') for ind in inds]
    
    def map_words_to_inds(self, tokens: List) -> List:
        return [self.word2ind.get(token, '') for token in tokens]

In [227]:
preprocessing = Preprocessing()

In [228]:
# Example part of a review
preprocessing.map_inds_to_words(train_data[0])[:10]

['the',
 'as',
 'you',
 'with',
 'out',
 'themselves',
 'powerful',
 'lets',
 'loves',
 'their']

### Load the stopwords and delete them from the reviews

In [229]:
nltk.download('stopwords')
_stopwords = stopwords.words('english')

# Check some english stopwords
_stopwords[:5]

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/pozdrowiony/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['i', 'me', 'my', 'myself', 'we']

#### Map inds to words

In [230]:
_train_data = [preprocessing.map_inds_to_words(rev) for rev in train_data]
_test_data = [preprocessing.map_inds_to_words(rev) for rev in test_data]

#### Remove stopwords

In [231]:
_train_data = [[token for token in rev if token not in _stopwords] for rev in _train_data]
_test_data = [[token for token in rev if token not in _stopwords] for rev in _test_data]

#### Map words to index

In [232]:
_train_data = [preprocessing.map_words_to_inds(rev) for rev in _train_data]
_test_data = [preprocessing.map_words_to_inds(rev) for rev in _test_data]

### Pad the sentences

In [233]:
MAX_LEN = len(max((_train_data + _test_data), key=len))

In [234]:
_train_data = sequence.pad_sequences(_train_data, maxlen=MAX_LEN)
_test_data = sequence.pad_sequences(_test_data, maxlen=MAX_LEN)

### Batch constants

In [235]:
BATCH_SIZE = 64
N_EPOCHS = 10

## Test various NN architectures

### Define neccessery elements

#### Constants

In [236]:
EMBEDDING_SIZE=32

# Define model paths
DENSE_PATH = './data/dense_net.h5'
DENSE_EMB_NOT_TRAIN_PATH = './data/dense_net_emb_not_train.h5'
DENSE_1_HIDDEN_PATH = './data/dense_net_1_hidden.h5'

LSTM_PATH = './data/lstm_net.h5'
GRU_PATH = './data/gru_net.h5'

CONV_1_LAYER_PATH = './data/conv_net_1_layer.h5'
CONV_1_LAYER_EMB_NOT_TRAIN_PATH = './data/conv_net_emb_not_train.h5'
CONV_1_LAYER_1_HIDDEN_PATH = './data/conv_net_1_1_hidden.h5'
CONV_2_LAYERS_PATH = './data/conv_net_2.h5'
CONV_2_LAYERS_1_HIDDEN_PATH = './data/conv_net_2_1_hidden.h5'

In [237]:
def create_and_train_net(model: Sequential, file_path: str,
                         override: bool = False) -> None:
    """ Create or import the net if created """
    
    model.compile(loss='binary_crossentropy', optimizer='adam',
                  metrics=['accuracy'])
    
    # Try to load the pretrained model
    if not os.path.isfile(file_path) or override:     
        model.fit(_train_data, train_labels,
                  validation_split=0.2,
                  batch_size=BATCH_SIZE,
                  epochs=N_EPOCHS)
        
        model.save(file_path)
    else:
        model = load_model(file_path)
        
    print(model.summary())

#### Define embedding layer constant for each architecture

In [238]:
EMBEDDING = Embedding(VOCAB_SIZE, EMBEDDING_SIZE, input_length=MAX_LEN)

#### Define list of the names of below nns and coresponding accuracies

In [239]:
nns = [
    ['dense', 0, 'YES', 0],
    ['dense', 0, 'NO', 0],
    ['dense', 1, 'YES', 0],
    ['lstm', 0, 'YES', 0],
    ['gru', 0, 'YES', 0],
    ['conv_1', 0, 'YES', 1],
    ['conv_1', 0, 'NO', 1],
    ['conv_1', 1, 'YES', 1],
    ['conv_2', 0, 'YES', 2],
    ['conv_2', 1, 'YES', 2],
]
columns = ['kind', 'n_hidden', 'emb_backprop', 'n_conv']

In [240]:
all_scores = []

### Dense

#### Only input and output layer

In [241]:
dense_net = Sequential()
dense_net.add(EMBEDDING)
dense_net.add(Flatten())
dense_net.add(Dense(1, activation='sigmoid'))

In [242]:
create_and_train_net(dense_net, DENSE_PATH)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 15s 747us/step - loss: 0.5103 - acc: 0.7607 - val_loss: 0.3052 - val_acc: 0.8718
Epoch 2/10
20000/20000 [==============================] - 6s 297us/step - loss: 0.2260 - acc: 0.9157 - val_loss: 0.2771 - val_acc: 0.8850
Epoch 3/10
20000/20000 [==============================] - 11s 551us/step - loss: 0.1421 - acc: 0.9535 - val_loss: 0.2839 - val_acc: 0.8866
Epoch 4/10
20000/20000 [==============================] - 8s 383us/step - loss: 0.0857 - acc: 0.9779 - val_loss: 0.3023 - val_acc: 0.8848
Epoch 5/10
20000/20000 [==============================] - 7s 351us/step - loss: 0.0483 - acc: 0.9926 - val_loss: 0.3262 - val_acc: 0.8798
Epoch 6/10
20000/20000 [==============================] - 8s 407us/step - loss: 0.0270 - acc: 0.9976 - val_loss: 0.3490 - val_acc: 0.8774
Epoch 7/10
20000/20000 [==============================] - 7s 348us/step - loss: 0.0156 - acc: 0.9994 - val_loss: 0.3718 -

In [243]:
scores = dense_net.evaluate(_test_data, test_labels, verbose=1)[1]
all_scores.append(scores)
scores

25000/25000 [==============================] - 3s 124us/step


0.8616

#### Only input and output layer without backpropagation on the embeddings

In [244]:
dense_net_emb_not_train = Sequential()
dense_net_emb_not_train.add(EMBEDDING)
dense_net_emb_not_train.add(Flatten())
dense_net_emb_not_train.add(Dense(1, activation='sigmoid'))

# Turn off the backprop on the embeddings
dense_net_emb_not_train.layers[0].trainable = False

In [245]:
create_and_train_net(dense_net_emb_not_train, DENSE_EMB_NOT_TRAIN_PATH)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 12s 624us/step - loss: 0.2139 - acc: 0.9818 - val_loss: 0.2915 - val_acc: 0.8802
Epoch 2/10
20000/20000 [==============================] - 3s 143us/step - loss: 0.0647 - acc: 0.9987 - val_loss: 0.3070 - val_acc: 0.8774
Epoch 3/10
20000/20000 [==============================] - 3s 143us/step - loss: 0.0353 - acc: 0.9997 - val_loss: 0.3251 - val_acc: 0.8772
Epoch 4/10
20000/20000 [==============================] - 3s 145us/step - loss: 0.0225 - acc: 1.0000 - val_loss: 0.3427 - val_acc: 0.8760
Epoch 5/10
20000/20000 [==============================] - 3s 143us/step - loss: 0.0156 - acc: 1.0000 - val_loss: 0.3593 - val_acc: 0.8742
Epoch 6/10
20000/20000 [==============================] - 3s 143us/step - loss: 0.0114 - acc: 1.0000 - val_loss: 0.3735 - val_acc: 0.8764
Epoch 7/10
20000/20000 [==============================] - 3s 144us/step - loss: 0.0087 - acc: 1.0000 - val_loss: 0.3882 - 

In [246]:
scores = dense_net_emb_not_train.evaluate(_test_data, test_labels, verbose=1)[1]
all_scores.append(scores)
scores

25000/25000 [==============================] - 3s 132us/step


0.86128

#### One hidden layer and dropout

In [247]:
OUTPUT_DENSE_HIDDEN = 100

In [248]:
dense_net_1_hidden = Sequential()
dense_net_1_hidden.add(EMBEDDING)
dense_net_1_hidden.add(Flatten())
dense_net_1_hidden.add(Dense(OUTPUT_DENSE_HIDDEN, activation='sigmoid'))
dense_net_1_hidden.add(Dropout(0.1))
dense_net_1_hidden.add(Dense(1, activation='sigmoid'))

In [249]:
create_and_train_net(dense_net_1_hidden, DENSE_1_HIDDEN_PATH)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 34s 2ms/step - loss: 0.0833 - acc: 0.9874 - val_loss: 0.3775 - val_acc: 0.8752
Epoch 2/10
20000/20000 [==============================] - 24s 1ms/step - loss: 0.0053 - acc: 1.0000 - val_loss: 0.4513 - val_acc: 0.8730
Epoch 3/10
20000/20000 [==============================] - 24s 1ms/step - loss: 0.0020 - acc: 1.0000 - val_loss: 0.4991 - val_acc: 0.8736
Epoch 4/10
20000/20000 [==============================] - 24s 1ms/step - loss: 0.0010 - acc: 1.0000 - val_loss: 0.5364 - val_acc: 0.8732
Epoch 5/10
20000/20000 [==============================] - 25s 1ms/step - loss: 6.3311e-04 - acc: 1.0000 - val_loss: 0.5648 - val_acc: 0.8734
Epoch 6/10
20000/20000 [==============================] - 24s 1ms/step - loss: 4.3157e-04 - acc: 1.0000 - val_loss: 0.5976 - val_acc: 0.8700
Epoch 7/10
20000/20000 [==============================] - 24s 1ms/step - loss: 3.1015e-04 - acc: 1.0000 - val_loss: 0.613

In [250]:
scores = dense_net_1_hidden.evaluate(_test_data, test_labels, verbose=1)[1]
all_scores.append(scores)
scores

25000/25000 [==============================] - 12s 465us/step


0.86116

### Recurrent

#### LSTM

In [251]:
lstm_net = Sequential()
lstm_net.add(EMBEDDING)
lstm_net.add(LSTM(100))
lstm_net.add(Dense(1, activation='sigmoid'))

In [252]:
create_and_train_net(lstm_net, LSTM_PATH)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 420s 21ms/step - loss: 0.2313 - acc: 0.9184 - val_loss: 0.3530 - val_acc: 0.8650
Epoch 2/10
20000/20000 [==============================] - 414s 21ms/step - loss: 0.1132 - acc: 0.9605 - val_loss: 0.3749 - val_acc: 0.8756
Epoch 3/10
20000/20000 [==============================] - 414s 21ms/step - loss: 0.2560 - acc: 0.8886 - val_loss: 0.3728 - val_acc: 0.8730
Epoch 4/10
20000/20000 [==============================] - 413s 21ms/step - loss: 0.0764 - acc: 0.9734 - val_loss: 0.3977 - val_acc: 0.8804
Epoch 5/10
20000/20000 [==============================] - 417s 21ms/step - loss: 0.0602 - acc: 0.9801 - val_loss: 0.4315 - val_acc: 0.8764
Epoch 6/10
20000/20000 [==============================] - 399s 20ms/step - loss: 0.0544 - acc: 0.9813 - val_loss: 0.4190 - val_acc: 0.8746
Epoch 7/10
20000/20000 [==============================] - 402s 20ms/step - loss: 0.0507 - acc: 0.9832 - val_loss: 0.4

In [253]:
scores = lstm_net.evaluate(_test_data, test_labels, verbose=1)[1]
all_scores.append(scores)
scores

25000/25000 [==============================] - 219s 9ms/step


0.85572

#### GRU

In [254]:
gru_net = Sequential()
gru_net.add(EMBEDDING)
gru_net.add(GRU(100))
gru_net.add(Dense(1, activation='sigmoid'))

In [255]:
create_and_train_net(gru_net, GRU_PATH)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 411s 21ms/step - loss: 0.1895 - acc: 0.9252 - val_loss: 0.3722 - val_acc: 0.8458
Epoch 2/10
20000/20000 [==============================] - 404s 20ms/step - loss: 0.0869 - acc: 0.9688 - val_loss: 0.3819 - val_acc: 0.8742
Epoch 3/10
20000/20000 [==============================] - 401s 20ms/step - loss: 0.0635 - acc: 0.9782 - val_loss: 0.4797 - val_acc: 0.8772
Epoch 4/10
20000/20000 [==============================] - 399s 20ms/step - loss: 0.0617 - acc: 0.9780 - val_loss: 0.4796 - val_acc: 0.8766
Epoch 5/10
20000/20000 [==============================] - 403s 20ms/step - loss: 0.0555 - acc: 0.9806 - val_loss: 0.4778 - val_acc: 0.8750
Epoch 6/10
20000/20000 [==============================] - 394s 20ms/step - loss: 0.0494 - acc: 0.9839 - val_loss: 0.5007 - val_acc: 0.8744
Epoch 7/10
20000/20000 [==============================] - 394s 20ms/step - loss: 0.0544 - acc: 0.9800 - val_loss: 0.4

In [256]:
scores = gru_net.evaluate(_test_data, test_labels, verbose=1)[1]
all_scores.append(scores)
scores

25000/25000 [==============================] - 176s 7ms/step


0.85868

### Conv

#### 1 Conv layer

In [257]:
conv_1_layer_net = Sequential()
conv_1_layer_net.add(EMBEDDING)
conv_1_layer_net.add(Conv1D(10, 5))
conv_1_layer_net.add(Flatten())
conv_1_layer_net.add(Dense(1, activation='sigmoid'))

In [258]:
create_and_train_net(conv_1_layer_net, CONV_1_LAYER_PATH)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 33s 2ms/step - loss: 0.1403 - acc: 0.9594 - val_loss: 0.4246 - val_acc: 0.8770
Epoch 2/10
20000/20000 [==============================] - 24s 1ms/step - loss: 0.0425 - acc: 0.9874 - val_loss: 0.4676 - val_acc: 0.8758
Epoch 3/10
20000/20000 [==============================] - 25s 1ms/step - loss: 0.0207 - acc: 0.9956 - val_loss: 0.5228 - val_acc: 0.8738
Epoch 4/10
20000/20000 [==============================] - 24s 1ms/step - loss: 0.0113 - acc: 0.9978 - val_loss: 0.5727 - val_acc: 0.8750
Epoch 5/10
20000/20000 [==============================] - 26s 1ms/step - loss: 0.0067 - acc: 0.9991 - val_loss: 0.6158 - val_acc: 0.8756
Epoch 6/10
20000/20000 [==============================] - 25s 1ms/step - loss: 0.0041 - acc: 0.9997 - val_loss: 0.6519 - val_acc: 0.8742
Epoch 7/10
20000/20000 [==============================] - 24s 1ms/step - loss: 0.0027 - acc: 1.0000 - val_loss: 0.6787 - val_acc:

In [259]:
scores = conv_1_layer_net.evaluate(_test_data, test_labels, verbose=1)[1]
all_scores.append(scores)
scores

25000/25000 [==============================] - 8s 325us/step


0.85812

#### 1 Conv layer without backpropagation on the embeddings

In [260]:
conv_1_layer_net_emb_not_train = Sequential()
conv_1_layer_net_emb_not_train.add(EMBEDDING)
conv_1_layer_net_emb_not_train.add(Conv1D(10, 5))
conv_1_layer_net_emb_not_train.add(Flatten())
conv_1_layer_net_emb_not_train.add(Dense(1, activation='sigmoid'))

# Turn off the backprop on the embeddings
conv_1_layer_net_emb_not_train.layers[0].trainable = False

In [261]:
create_and_train_net(conv_1_layer_net_emb_not_train, CONV_1_LAYER_EMB_NOT_TRAIN_PATH)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 37s 2ms/step - loss: 0.1450 - acc: 0.9593 - val_loss: 0.4297 - val_acc: 0.8768
Epoch 2/10
20000/20000 [==============================] - 24s 1ms/step - loss: 0.0440 - acc: 0.9864 - val_loss: 0.4660 - val_acc: 0.8746
Epoch 3/10
20000/20000 [==============================] - 24s 1ms/step - loss: 0.0209 - acc: 0.9955 - val_loss: 0.5205 - val_acc: 0.8742
Epoch 4/10
20000/20000 [==============================] - 24s 1ms/step - loss: 0.0110 - acc: 0.9981 - val_loss: 0.5728 - val_acc: 0.8746
Epoch 5/10
20000/20000 [==============================] - 23s 1ms/step - loss: 0.0063 - acc: 0.9992 - val_loss: 0.6119 - val_acc: 0.8740
Epoch 6/10
20000/20000 [==============================] - 23s 1ms/step - loss: 0.0039 - acc: 0.9996 - val_loss: 0.6406 - val_acc: 0.8730
Epoch 7/10
20000/20000 [==============================] - 25s 1ms/step - loss: 0.0026 - acc: 0.9999 - val_loss: 0.6826 - val_acc:

In [262]:
scores = conv_1_layer_net_emb_not_train.evaluate(_test_data, test_labels, verbose=1)[1]
all_scores.append(scores)
scores

25000/25000 [==============================] - 8s 332us/step


0.85812

#### 1 Conv layer + 1 hidden dense

In [263]:
conv_1_layer_1_hidden_net = Sequential()
conv_1_layer_1_hidden_net.add(EMBEDDING)
conv_1_layer_1_hidden_net.add(Conv1D(10, 5))
conv_1_layer_1_hidden_net.add(Flatten())
conv_1_layer_1_hidden_net.add(Dense(OUTPUT_DENSE_HIDDEN, activation='sigmoid'))
conv_1_layer_1_hidden_net.add(Dropout(0.1))
conv_1_layer_1_hidden_net.add(Dense(1, activation='sigmoid'))

In [264]:
create_and_train_net(conv_1_layer_1_hidden_net, CONV_1_LAYER_1_HIDDEN_PATH)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 43s 2ms/step - loss: 0.1056 - acc: 0.9617 - val_loss: 0.4275 - val_acc: 0.8734
Epoch 2/10
20000/20000 [==============================] - 34s 2ms/step - loss: 0.0156 - acc: 0.9970 - val_loss: 0.5299 - val_acc: 0.8702
Epoch 3/10
20000/20000 [==============================] - 31s 2ms/step - loss: 0.0030 - acc: 0.9997 - val_loss: 0.5834 - val_acc: 0.8712
Epoch 4/10
20000/20000 [==============================] - 31s 2ms/step - loss: 0.0011 - acc: 1.0000 - val_loss: 0.6616 - val_acc: 0.8718
Epoch 5/10
20000/20000 [==============================] - 30s 2ms/step - loss: 5.9345e-04 - acc: 1.0000 - val_loss: 0.7034 - val_acc: 0.8730
Epoch 6/10
20000/20000 [==============================] - 31s 2ms/step - loss: 3.6116e-04 - acc: 1.0000 - val_loss: 0.7406 - val_acc: 0.8720
Epoch 7/10
20000/20000 [==============================] - 33s 2ms/step - loss: 2.4818e-04 - acc: 1.0000 - val_loss: 0.770

In [265]:
scores = conv_1_layer_1_hidden_net.evaluate(_test_data, test_labels, verbose=1)[1]
all_scores.append(scores)
scores

25000/25000 [==============================] - 12s 461us/step


0.85884

#### 2 Conv layers

In [266]:
conv_2_layers_net = Sequential()
conv_2_layers_net.add(EMBEDDING)
conv_2_layers_net.add(Conv1D(2, 20))
conv_2_layers_net.add(Conv1D(10, 5))
conv_2_layers_net.add(Flatten())
conv_2_layers_net.add(Dense(1, activation='sigmoid'))

In [267]:
create_and_train_net(conv_2_layers_net, CONV_2_LAYERS_PATH)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 88s 4ms/step - loss: 0.1322 - acc: 0.9537 - val_loss: 0.4733 - val_acc: 0.8716
Epoch 2/10
20000/20000 [==============================] - 89s 4ms/step - loss: 0.0482 - acc: 0.9845 - val_loss: 0.5134 - val_acc: 0.8750
Epoch 3/10
20000/20000 [==============================] - 93s 5ms/step - loss: 0.0231 - acc: 0.9929 - val_loss: 0.6079 - val_acc: 0.8724
Epoch 4/10
20000/20000 [==============================] - 93s 5ms/step - loss: 0.0114 - acc: 0.9972 - val_loss: 0.6635 - val_acc: 0.8746
Epoch 5/10
20000/20000 [==============================] - 78s 4ms/step - loss: 0.0058 - acc: 0.9988 - val_loss: 0.7181 - val_acc: 0.8692
Epoch 6/10
20000/20000 [==============================] - 60s 3ms/step - loss: 0.0029 - acc: 0.9996 - val_loss: 0.7909 - val_acc: 0.8708
Epoch 7/10
20000/20000 [==============================] - 61s 3ms/step - loss: 0.0015 - acc: 0.9999 - val_loss: 0.8478 - val_acc:

In [268]:
scores = conv_2_layers_net.evaluate(_test_data, test_labels, verbose=1)[1]
all_scores.append(scores)
scores

25000/25000 [==============================] - 30s 1ms/step


0.85684

#### 2 Conv Layers + 1 hidden dense

In [269]:
conv_2_layers_1_hidden_net = Sequential()
conv_2_layers_1_hidden_net.add(EMBEDDING)
conv_2_layers_1_hidden_net.add(Conv1D(2, 20))
conv_2_layers_1_hidden_net.add(Conv1D(10, 5))
conv_2_layers_1_hidden_net.add(Flatten())
conv_2_layers_1_hidden_net.add(Dense(OUTPUT_DENSE_HIDDEN, activation='sigmoid'))
conv_2_layers_1_hidden_net.add(Dropout(0.1))
conv_2_layers_1_hidden_net.add(Dense(1, activation='sigmoid'))

In [270]:
create_and_train_net(conv_2_layers_1_hidden_net, CONV_2_LAYERS_1_HIDDEN_PATH)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 77s 4ms/step - loss: 0.1037 - acc: 0.9603 - val_loss: 0.4302 - val_acc: 0.8740
Epoch 2/10
20000/20000 [==============================] - 71s 4ms/step - loss: 0.0155 - acc: 0.9959 - val_loss: 0.5374 - val_acc: 0.8698
Epoch 3/10
20000/20000 [==============================] - 67s 3ms/step - loss: 0.0036 - acc: 0.9996 - val_loss: 0.6610 - val_acc: 0.8742
Epoch 4/10
20000/20000 [==============================] - 63s 3ms/step - loss: 0.0014 - acc: 0.9999 - val_loss: 0.6860 - val_acc: 0.8706
Epoch 5/10
20000/20000 [==============================] - 63s 3ms/step - loss: 5.0863e-04 - acc: 1.0000 - val_loss: 0.7585 - val_acc: 0.8734
Epoch 6/10
20000/20000 [==============================] - 63s 3ms/step - loss: 2.5013e-04 - acc: 1.0000 - val_loss: 0.8028 - val_acc: 0.8736
Epoch 7/10
20000/20000 [==============================] - 63s 3ms/step - loss: 1.6304e-04 - acc: 1.0000 - val_loss: 0.829

In [271]:
scores = conv_2_layers_1_hidden_net.evaluate(_test_data, test_labels, verbose=1)[1]
all_scores.append(scores)
scores

25000/25000 [==============================] - 39s 2ms/step


0.85488

## Summary

In [272]:
results = pd.DataFrame(nns)
results.columns = columns
results['accuracy'] = pd.Series(all_scores)
results.set_index('kind')

,n_hidden,emb_backprop,n_conv,accuracy
kind,,,,
dense,0,YES,0,0.86160
dense,0,NO,0,0.86128
dense,1,YES,0,0.86116
lstm,0,YES,0,0.85572
gru,0,YES,0,0.85868
conv_1,0,YES,1,0.85812
conv_1,0,NO,1,0.85812
conv_1,1,YES,1,0.85884
conv_2,0,YES,2,0.85684
